In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\keith\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\keith\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
#Import hate-speech annotated database
df = pd.read_csv('labeled_data.csv',header=None)
print(df)

       0                                                  1
0      2  !!! RT @mayasolovely: As a woman you shouldn't...
1      1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2      1  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3      1  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4      1  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...   ..                                                ...
24778  1  you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779  2  you've gone and broke the wrong heart baby, an...
24780  1  young buck wanna eat!!.. dat nigguh like I ain...
24781  1              youu got wild bitches tellin you lies
24782  2  ~~Ruffled | Ntac Eileen Dahlia - Beautiful col...

[24783 rows x 2 columns]


In [9]:
#Clean Data and Classes

newclass = []
for i in df[0]:
    if i == 0:
        newclass.append(1)
    else:
        newclass.append(i)
df[0] = newclass
#There is now no "hate speech" class and text is deemed as either offensive or inoffensive




In [ ]:
#Feature Engineering
#Data Splitting into training, testing and validation
#Train Model 


In [ ]:
#we changed the tweets into lower case. Also,
#we removed all the URLs, usernames, white spaces, hashtags,
#punctuations and stop-words using pattern matching
#techniques from the collected tweets
#We will use RE to pattern match to just return the tweets 
import re
def clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
    df[text_field] = df[text_field].apply(lambda elem: [word for word in word_tokenize(elem) if not word in stopwords.words()])
    return df

df = clean_text(df,1)



In [ ]:
print(df)